In [1]:
%pwd
%cd ../..

/home/rydevera3/data-science/umn_research/Deeplifting


In [2]:
import glob

import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME

In [3]:
# Get the results from deeplifting
files = glob.glob('./results/results-2023-08-03-2*')
deeplifting_results = pd.read_parquet(files)

# Select specific columns for results
deeplifting_results = (
    deeplifting_results[['problem_name', 'algorithm', 'x1', 'x2', 'f', 'total_time', 'global_minimum']]
)

# For now let's remove sine-envelope function because there is something
# going on there
mask = deeplifting_results['problem_name'] != 'sine_envelope'
deeplifting_results = deeplifting_results.loc[mask].reset_index(drop=True)
deeplifting_results['hits'] = np.abs(
    deeplifting_results['global_minimum'] - deeplifting_results['f']
) <= 1e-4
deeplifting_results['hits'] = deeplifting_results['hits'].astype(int)
deeplifting_results = deeplifting_results.rename(columns={'total_time': 'time'})
deeplifting_results = deeplifting_results.drop(columns=['global_minimum'])
deeplifting_results.head()

,problem_name,algorithm,x1,x2,f,time,hits
0,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,440.880969,1
1,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,3123.716541,1
2,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,112.328760,1
3,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,254.529593,1
4,kriging_peaks_red010,Deeplifting,-2.169612,0.295735,0.291121,2216.122304,1


In [4]:
# Load in the comparisons
comparison_files = glob.glob('./results/algorithm-comparisons-*.parquet')
comparison_results = pd.read_parquet(comparison_files)

# # Omit sine envelope function
# mask = comparison_results['problem_name'] != 'sine_envelope'
# comparison_results = comparison_results.loc[mask].reset_index(drop=True)
problems = [
    'ackley',
    'bukin_n6',
    'cross_in_tray',
    'drop_wave',
    'eggholder',
    'griewank',
    'holder_table',
    'levy',
    'levy_n13',
    'rastrigin',
    'schaffer_n2',
    'schaffer_n4',
    'schwefel',
    'shubert',
    'ex8_1_1',
    'kriging_peaks_red010',
    'mathopt6',
]

comparison_results = comparison_results.loc[comparison_results['problem_name'].isin(problems)]

In [5]:
comparison_results['problem_name'].value_counts()

problem_name
levy                    80
griewank                80
schaffer_n4             80
mathopt6                80
eggholder               80
bukin_n6                80
ex8_1_1                 80
holder_table            80
shubert                 80
schwefel                80
cross_in_tray           80
drop_wave               80
schaffer_n2             80
ackley                  80
kriging_peaks_red010    80
rastrigin               80
levy_n13                80
Name: count, dtype: int64

In [6]:
results_df = comparison_results.copy()

In [7]:
aggregated_results_df = (
    results_df
    .groupby(['problem_name', 'algorithm'])
    .agg({'hits': 'mean'})
    .rename(columns={'hits': 'Success Rate'})
    .unstack()
    .transpose()
    .reorder_levels(['algorithm', None])
    .transpose()
)

In [8]:
aggregated_results_df.mean().sort_values(ascending=False)

algorithm                           
Dual Annealing          Success Rate    0.829412
Differential Evolution  Success Rate    0.779412
PyGRANSO                Success Rate    0.179412
IPOPT                   Success Rate    0.132353
dtype: float64

In [9]:
aggregated_results_df

algorithm,Differential Evolution,Dual Annealing,IPOPT,PyGRANSO
,Success Rate,Success Rate,Success Rate,Success Rate
problem_name,,,,
ackley,1.00,1.00,0.05,0.00
bukin_n6,0.00,0.00,0.00,0.00
cross_in_tray,1.00,1.00,0.05,0.30
drop_wave,0.55,0.70,0.10,0.00
eggholder,0.20,0.45,0.00,0.05
ex8_1_1,1.00,1.00,0.95,0.85
griewank,0.75,0.55,0.00,0.00
holder_table,1.00,1.00,0.10,0.50


In [178]:
aggregated_results_df.shape

(12, 4)